# Query Decomposition

Sometimes a single question is multiple questions in disguise. For example: “Did Microsoft or Google make more money last year?”. To get to the correct answer for this seemingly simple question, we actually have to break it down: “How much money did Google make last year?” and “How much money did Microsoft make last year?”. Only if we know the answer to these 2 questions can we reason about the final answer.


Related Paper 
- [Least-to-Most Prompting Enables Complex Reasoning in Large Language Models](https://arxiv.org/pdf/2205.10625)

In [ ]:
!pip install langchain
!pip install langchain-groq
!pip install langchain-openai
!pip install langchain-core
!pip install langchain-community

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import OpenAIEmbeddings
from langchain_groq import ChatGroq

llm = ChatGroq(model="meta-llama/llama-4-scout-17b-16e-instruct")
embedding_model = OpenAIEmbeddings()

In [ ]:
template = """You are a helpful ai assistant who helps users to find relevant products from \
online clothing store by decomposing their query into 4 smaller-queries each of which addresses a \
specific clothing condition which then be used to query of database for retrieving relevant products. \
Carefully evaluate the query and come up with sub-queries that helps in recommending the best clothing items for the user.

Generate sub-queries related to: {question} \n
Stirctly respond with only the sub-questions separated by new line
"""
prompt_to_decompose = ChatPromptTemplate.from_template(template)

# Example User Input
user_input = "I am having a baby for the first time and I \
dont know what cloths should I get for her. \
I live in which is both very cold in winters and sunny in summers and also planning to go to outdoor walks with my baby."

l_to_m_decomposition_chain = (
    prompt_to_decompose
    | llm
    | StrOutputParser()
    | (lambda x: x.strip().split("\n"))
)
sub_queries = l_to_m_decomposition_chain.invoke(user_input)
print(sub_queries)

In [ ]:
template = """Here is the question you need to answer for reccomending unique clothing items from our store:

\n --- \n {question} \n --- \n

Here is any available background question + answer pairs:

\n --- \n {q_a_pairs} \n --- \n

Here is additional context relevant to the question: 

\n --- \n {context} \n --- \n

Use the above context and any background question + answer pairs to answer the question: \n {question}
"""

decomposition_prompt = ChatPromptTemplate.from_template(template)


def create_qa_pairs(question, answer):
    return f"Question: {question}\nAnswer: {answer}\n\n"

q_a_pairs = ""

for ques in sub_queries:
    rag_chain = (
        {'context': itemgetter("question") | vstore_retriever,
        "question": itemgetter("question"),
        "q_a_pairs": itemgetter("q_a_pairs")}
        | decomposition_prompt
        | llm
        | StrOutputParser()
    )
    answer = rag_chain.invoke({"question":ques, "q_a_pairs":q_a_pairs})
    q_a_pair = create_qa_pairs(ques, answer)
    q_a_pairs += "\n---\n" + q_a_pair

print(answer)